In [1]:
source("/secure/projects/HTAPP_MBC/src/init_phase2.R")
library(infercnv)

Registered S3 method overwritten by 'R.oo':
  method        from       
  throw.default R.methodsS3
Loading combined annotation: annot


In [2]:
run_infercnv = function(run_sample){
    
    wd=file.path(analysisDir,"00_infercnv",run_sample)
    suppressWarnings(dir.create(wd,recursive = TRUE))
    setwd(wd)
    if (file.exists("out/infercnv.png")){
        print("Already done.")
        return()}
    suppressWarnings(rm(ss))
    simpleCache(cacheName = run_sample,cacheDir = cacheDir,cacheSubDir = "filtered",assignToVariable = "ss")
    
    rc_matrix=as.matrix(ss@assays$RNA@counts)
    annotations=as.data.table(ss@meta.data[,"labels_cl_unif",drop=FALSE],keep.rownames = "cellid")
    annotations[,N_ct:=.N,by=labels_cl_unif]
    annotations[N_ct<20,labels_cl_unif:="others",]
    ref_group_names=unique(annotations[N_ct>20&labels_cl_unif%in%c("DC","Monocyte","Endothelial","Macrophage","Fibroblast","NK"),]$labels_cl_unif)
    annotation_filt=annotations[,c("cellid","labels_cl_unif"),with=FALSE]
    
    
    sink("ref_stats.txt")
    print(table(annotations$labels_cl_unif))
    print(ref_group_names)
    sink()
    
    write.table(annotation_filt,paste0(wd,"/annot.tsv"),row.names=FALSE,col.names = FALSE,sep="\t",quote=FALSE)
    
    # create the infercnv object
    invisible(capture.output(
        infercnv_obj <- CreateInfercnvObject(raw_counts_matrix=rc_matrix,
                                    annotations_file="annot.tsv",
                                    delim="\t",
                                    gene_order_file=file.path(extDir,"gencode_v19_gene_pos.txt"),
                                    ref_group_names=ref_group_names)      
    ))

    # perform infercnv operations to reveal cnv signal
    invisible(capture.output(
        infercnv_obj <- infercnv::run(infercnv_obj,num_threads = 6, #usualy 6
                             cutoff=0.1,  # use 1 for smart-seq, 0.1 for 10x-genomics
                             out_dir="out",  # dir is auto-created for storing outputs
                             cluster_by_groups=T,   # cluster
                             denoise=T,
                             HMM=F,no_plot=F)
    ))
}

In [ ]:
for (sample in c("HTAPP-231-SMP-6758","HTAPP-862-SMP-7059","HTAPP-895-SMP-7359","HTAPP-947-SMP-7509")){
    if (file.exists(paste0(cacheDir,"/filtered/",sample,".RData"))){
        print(sample)
        suppressMessages(run_infercnv(sample))
        }
}

In [ ]:
for (sample in grep("HTAPP-214-SMP-6753|iso",c(unique(sample_sheet$sampleid),unique(sample_sheet$channel_id)),invert = TRUE,value = TRUE)){
    if (file.exists(paste0(cacheDir,"/filtered/",sample,".RData"))){
        print(sample)
        suppressMessages(run_infercnv(sample))
        }
}

[1] "HTAPP-254-SMP-571"
[1] "Already done."
[1] "HTAPP-806-SMP-6800"
[1] "Already done."
[1] "HTAPP-285-SMP-751"
[1] "Already done."
[1] "HTAPP-309-SMP-871"
[1] "Already done."
[1] "HTAPP-364-SMP-1321"
[1] "Already done."
[1] "HTAPP-423-SMP-1741"
[1] "Already done."
[1] "HTAPP-425-SMP-1771"
[1] "Already done."
[1] "HTAPP-382-SMP-1441"
[1] "Already done."
[1] "HTAPP-394-SMP-1561"
[1] "Already done."
[1] "HTAPP-414-SMP-1681"
[1] "Already done."
[1] "HTAPP-562-SMP-2581"
[1] "Already done."
[1] "HTAPP-589-SMP-2851"
[1] "Already done."
[1] "HTAPP-600-SMP-2941"
[1] "Already done."
[1] "HTAPP-735-SMP-3841"
[1] "Already done."
[1] "HTAPP-745-SMP-3961"
[1] "Already done."
[1] "HTAPP-749-SMP-3991"
[1] "Already done."
[1] "HTAPP-752-SMP-4051"
[1] "Already done."
[1] "HTAPP-783-SMP-4081"
[1] "Already done."
[1] "HTAPP-851-SMP-4351"
[1] "Already done."
[1] "HTAPP-321-SMP-1021"
[1] "Already done."
[1] "HTAPP-917-SMP-4531"
[1] "Already done."
[1] "HTAPP-963-SMP-4741"
[1] "Already done."
[1] "HTAPP-85

In [ ]:
1+1